# MapRerankDocumentsChain

## 源码分析

### 类和属性定义部分

#### 类定义

In [ ]:
class MapRerankDocumentsChain(BaseCombineDocumentsChain):
    """Combining documents by mapping a chain over them, then reranking results."""

`MapRerankDocumentsChain` 是一个继承自 `BaseCombineDocumentsChain` 的类。它的主要作用是对一组文档逐一应用链，然后根据输出的得分对结果进行重新排序，最终返回得分最高的结果。

#### 属性定义

In [ ]:
    llm_chain: LLMChain
    document_variable_name: str
    rank_key: str
    answer_key: str
    metadata_keys: Optional[List[str]] = None
    return_intermediate_steps: bool = False

- `llm_chain`: 用于处理每个文档的链（例如 `LLMChain`）。
- `document_variable_name`: 在 `llm_chain` 中用于存储文档内容的变量名。
- `rank_key`: 用于对结果进行排序的键（例如 `score`）。
- `answer_key`: 最终返回的答案对应的键。
- `metadata_keys`: 附加的元数据键，如果需要的话，可以将文档的某些元数据与结果一起返回。
- `return_intermediate_steps`: 是否返回中间步骤的结果。

### Schema生成部分

`get_output_schema` 方法

In [ ]:
    def get_output_schema(self, config: Optional[RunnableConfig] = None) -> Type[BaseModel]:
        schema: Dict[str, Any] = {
            self.output_key: (str, None),
        }
        if self.return_intermediate_steps:
            schema["intermediate_steps"] = (List[str], None)
        if self.metadata_keys:
            schema.update({key: (Any, None) for key in self.metadata_keys})

        return create_model("MapRerankOutput", **schema)

- `get_output_schema`: 根据是否返回中间步骤和元数据，动态生成输出的 `schema`。这个 `schema` 会在处理结果时被用来定义输出的数据结构。

### 输入验证部分

#### `validate_llm_output `方法

In [ ]:
    @root_validator(pre=False, skip_on_failure=True)
    def validate_llm_output(cls, values: Dict) -> Dict:
        output_parser = values["llm_chain"].prompt.output_parser
        if not isinstance(output_parser, RegexParser):
            raise ValueError(
                "Output parser of llm_chain should be a RegexParser,"
                f" got {output_parser}"
            )
        output_keys = output_parser.output_keys
        if values["rank_key"] not in output_keys:
            raise ValueError(
                f"Got {values['rank_key']} as key to rank on, but did not find "
                f"it in the llm_chain output keys ({output_keys})"
            )
        if values["answer_key"] not in output_keys:
            raise ValueError(
                f"Got {values['answer_key']} as key to return, but did not find "
                f"it in the llm_chain output keys ({output_keys})"
            )
        return values

- `validate_llm_output`: 确保 `llm_chain` 的输出解析器是 `RegexParser`，并验证 `rank_key` 和 `answer_key` 是否在输出键中。如果不符合要求，则抛出相应的错误。

### 处理输入部分

#### `get_default_document_variable_name`方法

In [ ]:
    @root_validator(pre=True)
    def get_default_document_variable_name(cls, values: Dict) -> Dict:
        if "llm_chain" not in values:
            raise ValueError("llm_chain must be provided")

        llm_chain_variables = values["llm_chain"].prompt.input_variables
        if "document_variable_name" not in values:
            if len(llm_chain_variables) == 1:
                values["document_variable_name"] = llm_chain_variables[0]
            else:
                raise ValueError(
                    "document_variable_name must be provided if there are "
                    "multiple llm_chain input_variables"
                )
        else:
            if values["document_variable_name"] not in llm_chain_variables:
                raise ValueError(
                    f"document_variable_name {values['document_variable_name']} was "
                    f"not found in llm_chain input_variables: {llm_chain_variables}"
                )
        return values

- `get_default_document_variable_name`: 验证 `document_variable_name` 是否正确设置。如果未提供且 `llm_chain` 只有一个输入变量，则默认使用该输入变量名。

### 文档处理部分

#### `combine_docs`方法

In [ ]:
    def combine_docs(
        self, docs: List[Document], callbacks: Callbacks = None, **kwargs: Any
    ) -> Tuple[str, dict]:
        results = self.llm_chain.apply_and_parse(
            [{**{self.document_variable_name: d.page_content}, **kwargs} for d in docs],
            callbacks=callbacks,
        )
        return self._process_results(docs, results)

- `combine_docs`: 对每个文档应用 `llm_chain` 并解析结果，然后调用 `_process_results` 方法处理这些结果。最终返回最高得分的答案和相关的附加信息。

#### `acombine_docs`方法

In [ ]:
    async def acombine_docs(
        self, docs: List[Document], callbacks: Callbacks = None, **kwargs: Any
    ) -> Tuple[str, dict]:
        results = await self.llm_chain.aapply_and_parse(
            [{**{self.document_variable_name: d.page_content}, **kwargs} for d in docs],
            callbacks=callbacks,
        )
        return self._process_results(docs, results)

- `acombine_docs`: 异步版本的 `combine_docs`，对文档异步应用链并处理结果。

#### `_process_results`方法

In [ ]:
    def _process_results(
        self,
        docs: List[Document],
        results: Sequence[Union[str, List[str], Dict[str, str]]],
    ) -> Tuple[str, dict]:
        typed_results = cast(List[dict], results)
        sorted_res = sorted(
            zip(typed_results, docs), key=lambda x: -int(x[0][self.rank_key])
        )
        output, document = sorted_res[0]
        extra_info = {}
        if self.metadata_keys is not None:
            for key in self.metadata_keys:
                extra_info[key] = document.metadata[key]
        if self.return_intermediate_steps:
            extra_info["intermediate_steps"] = results
        return output[self.answer_key], extra_info

`_process_results`: 将 `llm_chain` 的结果按 `rank_key` 排序，并返回得分最高的答案。同时，附加返回任何指定的元数据键和中间步骤的结果。

### 类型定义

#### `_chain_type`属性

In [ ]:
    @property
    def _chain_type(self) -> str:
        return "map_rerank_documents_chain"

- `_chain_type`: 返回链的类型，表明该链是一个 map_rerank_documents_chain.

### 总结

`MapRerankDocumentsChain` 类用于处理一组文档，通过 `llm_chain` 对每个文档进行处理，然后根据结果进行重新排序。最终，它返回得分最高的答案，并可以选择返回文档的元数据和中间步骤的结果。这种方法特别适合需要从一组可能答案中挑选出最佳答案的场景。

## demo

In [57]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers.regex import RegexParser
from langchain_community.chat_models import ChatTongyi
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.map_rerank import MapRerankDocumentsChain
from langchain_core.documents import Document
import os

In [58]:

# 定义新的提示模板
prompt_template = (
    "Use the following context to answer the question: '{question}'. "
    "Provide your answer in the format 'Answer: [your answer]' and a confidence score "
    "in the format 'Score: [your score]'. Context: {context}"
)


# 调整后的正则表达式，提取数字部分
output_parser = RegexParser(
    regex=r"Answer: (.*?)\nScore: (\d+)%",  # 提取数字部分的分数
    output_keys=["answer", "score"],
)

# 创建PromptTemplate对象
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)


In [59]:

# 从环境变量中获取API密钥，用于初始化 ChatTongyi 模型
api_key = os.getenv("KEY_TONGYI")
if not api_key:
    raise ValueError("API Key is not set. Please ensure that the 'KEY_TONGYI' environment variable is set.")


# 初始化 ChatTongyi 模型，设置文本生成的温度参数，温度越低生成的文本越接近输入
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0,  # 设置生成文本的倾向，值越小生成的文本越接近输入
    streaming=True
)


In [60]:
# LLM链
llm_chain = LLMChain(
  llm=llm,
  prompt=prompt
)

In [65]:
# 创建 MapRerankDocumentsChain
chain = MapRerankDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    rank_key="score",    # 使用 'score' 作为排序依据
    answer_key="answer",  # 返回 'answer' 作为最终结果
    return_intermediate_steps=True
)

In [63]:
from langchain.docstore.document import Document

# 定义文档
docs = [
    Document(page_content="Water is composed of two hydrogen atoms and one oxygen atom."),
    Document(page_content="H2O is the chemical formula for water."),
    Document(page_content="Water is made up of H and O."),
]

# 运行链，提取最高得分的答案
result, _ = chain.combine_docs(docs, question="What is the chemical formula for water?")
print(result)  # 输出得分最高的答案


e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


H2O


In [66]:
# 运行链，提取最高得分的答案
result, other_answer = chain.combine_docs(docs, question="水是有哪几个元素组成的？")
# print(result)  # 输出得分最高的答案

e:\AAAAWork\python\LLM_RAG\demo\lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [70]:
result


'Water is composed of two hydrogen atoms and one oxygen atom.'

In [71]:
other_answer

{'intermediate_steps': [{'answer': 'Water is composed of two hydrogen atoms and one oxygen atom.',
   'score': '100'},
  {'answer': 'Water is composed of two hydrogen (H) and one oxygen (O) element.',
   'score': '100'},
  {'answer': 'Water is composed of hydrogen (H) and oxygen (O). ',
   'score': '100'}]}